In [6]:
%load_ext autoreload

import sys
sys.path.insert(0, "../src-py")

import json
import pandas as pd
pd.set_option('display.max_colwidth', None)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
data_path = '/workspace/ceph_data/belief-based-argumentation-generation-cleaned/data'

### Loading data:

In [8]:
users_info = json.load(open(data_path+ '/raw_data/users.json'))
debates = json.load(open(data_path+'/raw_data/debates.json'))

In [9]:
big_issues_names = list(users_info['Skeptikitten']['big_issues_dict'].keys())
users_big_issues = [list(value['big_issues_dict'].values()) for key, value in users_info.items()]

In [10]:
big_issues_df = pd.DataFrame(users_big_issues, columns=big_issues_names)

In [11]:
big_issues_df.head()

,Abortion,Affirmative Action,Animal Rights,Barack Obama,Border Fence,Capitalism,Civil Unions,Death Penalty,Drug Legalization,Electoral College,...,Social Programs,Social Security,Socialism,Stimulus Spending,Term Limits,Torture,United Nations,War in Afghanistan,War on Terror,Welfare
0,N/S,N/S,N/S,N/S,N/S,N/S,N/S,N/S,N/S,N/S,...,N/S,N/S,N/S,N/S,N/S,N/S,N/S,N/S,N/S,N/S
1,N/S,N/S,N/S,N/S,N/S,N/S,N/S,N/S,N/S,N/S,...,N/S,N/S,N/S,N/S,N/S,N/S,N/S,N/S,N/S,N/S
2,Pro,N/O,Pro,Pro,N/O,Pro,N/O,Und,Und,Und,...,Pro,Pro,Con,Pro,Pro,Con,Pro,Pro,Pro,Pro
3,Pro,N/S,N/S,N/S,N/S,N/S,N/S,N/S,N/S,N/S,...,N/S,N/S,N/S,N/S,N/S,N/S,N/S,N/S,N/S,N/S
4,N/S,N/S,N/S,N/S,N/S,N/S,N/S,N/S,N/S,N/S,...,N/S,N/S,N/S,N/S,N/S,N/S,N/S,N/S,N/S,N/S


In [12]:
big_issues_dict={}
for big_issue in big_issues_names:
    s = big_issues_df[big_issue].value_counts()
    big_issues_dict[big_issue] = (s['Pro'] + s['Con'])/len(big_issues_df)

In [13]:
sorted(big_issues_dict.items(), key=lambda x: -x[1])

[('Abortion', 0.3015568492546529),
 ('Death Penalty', 0.2823057246185058),
 ('Gay Marriage', 0.27286760165828705),
 ('Drug Legalization', 0.26481873511510984),
 ('Gun Rights', 0.24521478345241246),
 ('Global Warming Exists', 0.23659257299109113),
 ('Environmental Protection', 0.23498279968245567),
 ('Medical Marijuana', 0.23313045779306696),
 ('Smoking Ban', 0.2192378936226515),
 ('Minimum Wage', 0.2071315162741466),
 ('Border Fence', 0.20514686424980152),
 ('War on Terror', 0.20435300344006352),
 ('National Health Care', 0.20020728587809827),
 ('War in Afghanistan', 0.19409896798094733),
 ('United Nations', 0.19136455852518303),
 ('Racial Profiling', 0.1888947693393314),
 ('Welfare', 0.18558701596542296),
 ('Iran-Iraq War', 0.18300696833377436),
 ('Medicaid & Medicare', 0.17698685719326102),
 ('Social Security', 0.17681044367998588),
 ('Animal Rights', 0.1733703801711211),
 ('Civil Unions', 0.16986416159477816),
 ('Term Limits', 0.16263120755049837),
 ('Euthanasia', 0.1613742612684131

### Building Big-Issues BOWs:

In [14]:
%autoreload

from argument_retrieval import *

amar = ArgsMeArgumentRetriever(keyphrases=True,
                            aspect_occurances=True,
                            synset=False,
                            topic_signatures=True,
                            argsme_corpus_path=data_path+'/argsme_data/args-me.json',
                            sample_size=1000)

In [16]:
big_issues_df = pd.read_pickle(data_path+'/preprocessed_data/big_issues_df.pkl')

In [192]:
big_issues_bows = [amar.extract_topic_pro_con_signatures(topic, 1, 60, 10, 25)
                   for topic in big_issues_df['topic'].tolist()]

CALLING:  https://www.args.me/api/v2/arguments?query=abortion&pageSize=60
Number of retrieved arguments:  60
Number of pro arguments: 25
Number of con arguments: 35
CALLING:  https://www.args.me/api/v2/arguments?query=affirmative+action&pageSize=60
Number of retrieved arguments:  60
Number of pro arguments: 33
Number of con arguments: 27
CALLING:  https://www.args.me/api/v2/arguments?query=animal+rights&pageSize=60
Number of retrieved arguments:  60
Number of pro arguments: 33
Number of con arguments: 26
CALLING:  https://www.args.me/api/v2/arguments?query=barack+obama&pageSize=60
Number of retrieved arguments:  60
Number of pro arguments: 34
Number of con arguments: 26
CALLING:  https://www.args.me/api/v2/arguments?query=border+fence&pageSize=60
Number of retrieved arguments:  60
Number of pro arguments: 26
Number of con arguments: 34
CALLING:  https://www.args.me/api/v2/arguments?query=capitalism&pageSize=60
Number of retrieved arguments:  60
Number of pro arguments: 30
Number of con

In [198]:
big_issues_pro_bows = [x[0] for x in big_issues_bows]
big_issues_con_bows = [x[1] for x in big_issues_bows]

In [207]:
big_issues_df['pro_bow'] = big_issues_pro_bows
big_issues_df['con_bow'] = big_issues_con_bows
big_issues_df[big_issues_df.topic.isin(['Abortion', 'Death Penalty', 'Gay Marriage', 'Drug Legalization', 'Global Warming Exists',
          'Environmental Protection', 'Medical Marijuana', 'Smoking Ban', 'Minimum Wage', 'Border Fence'])].head(n=10)

,topic,user_percentage,pro_bow,con_bow
0,Abortion,0.301557,"[abortion, fetus, mother, baby, aborted, embyro, right, murder, alive, illegal, choice, performed, procedures, justified, parents, option, organs, considered, goes, personhood, hippocratic, homemade, contraceptive, procedure, circumstances]","[accessed, contraception, miscarriage, termed, delivery, rate, abort, weeks, conception, babies, commonly, getting, stress, doubles, ptsd, available, increased, psychological, pelvic, rape, sanctity, saline, defiance, online, inflammatory]"
4,Border Fence,0.205147,"[illegal, immigrants, immigration, mexico, illegals, aliens, country, crossing, percent, crime, counties, secure, jobs, illegally, states, citizens, effective, arizona, apprehensions, million, america, california, trafficking, southern, guards]","[migrants, environmental, varrack, tinyurl, ineffective, build, hurts, bears, report, penn, increase, actually, foot, harms, teller, erosion, migrate, endangered, square, impact, nytimes, economy, perry, nonborder, fiscal]"
7,Death Penalty,0.282306,"[crime, murder, voluntary, statistic, criminals, crimes, states, defendant, rate, current, murderers, affirming, singapore, executing, texas, commit, convict, innocents, executions, norway, homicide, criminal, kronenwetter, committing, variables]","[lwop, million, plea, capital, convicted, retribution, killing, bargains, revenge, prisoners, dollars, evidence, someone, murdered, wrongly, people, incapacitation, victim, median, procon, punishments, prosecutors, dudley, counted, guilty]"
8,Drug Legalization,0.264819,"[drug, legalization, prohibition, thett, alcohol, decriminalization, legalizing, market, harm, criminalization, people, would, jail, abuse, harmful, dangerous, illicit, cartels, psychedelics, consumption, substance, penalties, treatment, enforcement, regulation]","[danielle, costs, decrease, addictive, percent, rand, colorado, number, addicted, rehab, massively, selling, persuasion, libertarian, tolerance, workplace, advocating, corporations, tens, amsterdam, hooked, nber, barriers, metric, drugabuse]"
10,Environmental Protection,0.234983,"[environmental, extraction, resource, protection, environment, developing, resources, countries, country, prioritize, conflict, pollution, prioritization, water, forest, natural, money, wealth, deforestation, curse, benefits, albania, green, protecting, degradation]","[integrated, morality, achieve, glaciers, foundation, policymakers, strengthening, technologies, evidence, upholds, hurts, subpoint, extractive, shelves, glacier, demonstrate, heavily, observations, research, economies, curve, pair, empirical, climatologists, inverse]"
17,Gay Marriage,0.272868,"[couples, marry, gays, married, straight, children, legal, couple, churches, rights, religious, legalized, polygamy, orientation, illegal, homosexual, parents, equal, people, recognizing, allowing, homosexuals, adopt, inequality, recognized]","[offspring, chair, unnatural, create, promoted, females, immoral, bond, males, exclusivity, sterile, biological, rates, standing, procreate, incestuous, lesbians, standpoint, family, opponent, wife, purpose, defeats, perverse, promoting]"
18,Global Warming Exists,0.236593,"[carbon, atmosphere, dioxide, temperatures, greenhouse, graph, earth, gases, heat, data, levels, surface, increase, radiation, infrared, effect, nasa, exists, increased, result, methane, amount, skeptics, decade, increasing]","[models, winter, predicted, phil, moore, jones, predictions, artic, hockey, dataset, conduct, garbage, skeptical, dailymail, report, faulty, measuring, gore, ranked, since, onus, pseudoscience, lovelock, hurricanes, probandi]"
28,Medical Marijuana,0.233130,"[medical, cancer, pain, patients, cannabis, drugs, doctors, user, alzheimer, addictive, glaucoma, legalized, prescription, disease, epilepsy, chemotherapy, illegal, harmful, nausea, prescribed, definite, studies, cells, legal, help]","[immune, percent, pulmonary, increased, linked, irritability, tob

In [201]:
big_issues_df.to_csv(data_path+'/big_issues_bows/big_issues_new_bows.csv')
big_issues_df.to_pickle(data_path+'/big_issues_bows/big_issues_new_bows.pkl')